# 使用 Optuna 进行超参数调优

本教程的 Github 仓库: https://github.com/araffin/tools-for-robotic-rl-icra2022

相关链接:
- Optuna: https://github.com/optuna/optuna
- Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3
- 官方文档: https://stable-baselines3.readthedocs.io/en/master/
- SB3 Contrib: https://github.com/Stable-Baselines-Team/stable-baselines3-contrib
- RL Baselines3 zoo: https://github.com/DLR-RM/rl-baselines3-zoo

[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) 是一个使用 Stable-Baselines3 预训练的强化学习智能体的集合。

它还提供了用于训练、评估智能体、调优超参数和录制视频的基础脚本。

## 引言

在本 Notebook 中，你将学习到调优超参数的重要性。你将首先尝试手动优化参数，然后我们将看到如何使用 Optuna 自动化搜索过程。

## 为本地运行设置环境 (建议)

为了避免包版本冲突，强烈建议你创建一个独立的虚拟环境。你可以使用 `conda` 或 `venv`。

**使用 Conda:**
```bash
conda create -n optuna_lab python=3.9
conda activate optuna_lab
```

**使用 venv:**
```bash
python -m venv optuna_lab
source optuna_lab/bin/activate  # 在 Windows 上使用 `optuna_lab\Scripts\activate`
```

创建并激活环境后，你可以运行下面的安装命令来安装所有必需的依赖项。

## 安装依赖

In [ ]:
# 安装 Stable-Baselines3 核心库
!pip install stable-baselines3

In [ ]:
# 可选：安装 SB3 contrib 以使用额外的算法
!pip install sb3-contrib

In [ ]:
# 安装 Optuna，我们将在最后一部分用它来进行超参数调优
!pip install optuna

## 导入库

In [ ]:
import gym
import numpy as np

首先你需要导入强化学习模型，请查阅官方文档以了解在什么问题上可以使用哪些模型。

In [ ]:
from stable_baselines3 import PPO, A2C, SAC, TD3, DQN

In [ ]:
# 来自 Contrib 仓库的算法
# https://github.com/Stable-Baselines-Team/stable-baselines3-contrib
from sb3_contrib import QRDQN, TQC

In [ ]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

# 第一部分：调优超参数的重要性



与监督学习相比，深度强化学习对超参数（如学习率、神经元数量、层数、优化器等）的选择要敏感得多。

糟糕的超参数选择可能导致模型收敛效果差或不稳定。此外，由于随机种子（用于初始化网络权重和环境）的不同，模型性能的差异性也加剧了这一挑战。

除了超参数，选择合适的算法也是一个重要的决定。我们将在简单的“倒立摆”任务（Pendulum）上进行演示。

参考 [gym 文档](https://gym.openai.com/envs/Pendulum-v0/)：“倒立摆上摆问题是控制文献中的一个经典问题。在这个版本的问题中，倒立摆从一个随机位置开始，目标是将其向上摆动并保持直立。”


我们先来试试 PPO 算法，并给它一个 4000 步（约 20 个回合）的少量训练预算：

In [ ]:
env_id = "Pendulum-v1"
# 这个环境只用于评估
eval_envs = make_vec_env(env_id, n_envs=10)
# 4000 个训练时间步
budget_pendulum = 4000

### PPO 算法

In [ ]:
ppo_model = PPO("MlpPolicy", env_id, seed=0, verbose=0).learn(budget_pendulum)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"PPO 平均回合奖励: {mean_reward:.2f} +/- {std_reward:.2f}")

### A2C 算法 (练习)

In [ ]:
# 定义并训练一个 A2C 模型
a2c_model = A2C("MlpPolicy", env_id, seed=0, verbose=0).learn(budget_pendulum)

In [ ]:
# 评估训练好的 A2C 模型
mean_reward, std_reward = evaluate_policy(a2c_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"A2C 平均回合奖励: {mean_reward:.2f} +/- {std_reward:.2f}")

两种算法的得分都远未解决这个问题（平均奖励约为 -200 才算解决）。
现在，我们来试试离线策略（off-policy）算法：

### 延长 PPO 的训练时间？

也许训练更长时间会有帮助？

你可以尝试将训练预算增加10倍，但对于 A2C/PPO 来说，延长训练时间帮助不大，真正需要的是找到更好的超参数。

In [ ]:
# 延长训练时间
new_budget = 10 * budget_pendulum

ppo_model_long = PPO("MlpPolicy", env_id, seed=0, verbose=0).learn(new_budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model_long, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"PPO (长时训练) 平均回合奖励: {mean_reward:.2f} +/- {std_reward:.2f}")

### PPO - 调优后的超参数

实际上，我们可以使用 Optuna 来调优超参数，并找到一个有效的解决方案（来自 [RL Zoo](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/ppo.yml)）:

In [ ]:
tuned_params = {
    "gamma": 0.9,
    "use_sde": True,
    "sde_sample_freq": 4,
    "learning_rate": 1e-3,
}

ppo_tuned_model = PPO("MlpPolicy", env_id, seed=1, verbose=1, **tuned_params).learn(50_000, log_interval=5)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_tuned_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"调优后的 PPO 平均回合奖励: {mean_reward:.2f} +/- {std_reward:.2f}")

注意：如果你在简单的 `MountainCarContinuous` 环境上尝试 SAC 算法，而不使用调优过的超参数，你会遇到一些问题：https://github.com/rail-berkeley/softlearning/issues/76

即便是简单的环境，对于 SOTA (State-of-the-art) 算法来说也可能具有挑战性。

# 第二部分：研究生梯度下降 (手动调参)


### 挑战 (10分钟): "研究生梯度下降"
这个挑战是在有限的 20,000 个训练步数的预算下，为 A2C 算法在 `CartPole-v1` 环境中找到最佳的超参数（以获得最高性能）。

`CartPole-v1` 的最高奖励是 500。

你找到的超参数应该在不同的随机种子下都能表现良好。

In [ ]:
budget = 20_000

#### 基线: 默认超参数

In [ ]:
eval_envs_cartpole = make_vec_env("CartPole-v1", n_envs=10)

In [ ]:
model = A2C("MlpPolicy", "CartPole-v1", seed=8, verbose=0).learn(budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_envs_cartpole, n_eval_episodes=50, deterministic=True)

print(f"平均奖励:{mean_reward:.2f} +/- {std_reward:.2f}")

**你的目标是超越这个基线性能，并尽可能接近 500 的最优分数**

开始调参吧！

In [ ]:
import torch.nn as nn

In [ ]:
policy_kwargs = dict(
    # 为 actor/critic 定义网络结构
    net_arch=[
      dict(vf=[64, 64], pi=[64, 64]),
    ],
    # 激活函数
    activation_fn=nn.Tanh,
)

# 尝试调整这些超参数！
hyperparams = dict(
    n_steps=7, # 在更新策略前收集数据的步数
    learning_rate=7e-4,
    gamma=0.99, # 折扣因子
    max_grad_norm=0.4, # 梯度裁剪的最大值
    ent_coef=0.0, # 损失计算中的熵系数
)

model = A2C("MlpPolicy", "CartPole-v1", seed=8, verbose=0, policy_kwargs=policy_kwargs, **hyperparams).learn(budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_envs_cartpole, n_eval_episodes=50, deterministic=True)

print(f"手动调参后的平均奖励:{mean_reward:.2f} +/- {std_reward:.2f}")

#### 提示 - 推荐的超参数范围

这里是一些在自动调参时常用的参数范围，可以给你一些手动调参的灵感。

```python
# 折扣因子 gamma
gamma = trial.suggest_float("gamma", 0.9, 0.99999, log=True)
# 梯度裁剪
max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
# 更新步数 (从 2**3=8 到 2**10=1024)
n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
# 学习率
learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
# 熵系数
ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)

# 网络结构
# net_arch tiny: {"pi": [64], "vf": [64]}
# net_arch default: {"pi": [64, 64], "vf": [64, 64]}
# 激活函数
# activation_fn = nn.Tanh / nn.ReLU
```

# 第三部分：自动超参数调优


在这一部分，我们将创建一个脚本来自动搜索最佳超参数。

### 导入 Optuna 相关库

In [ ]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

### 配置参数

In [ ]:
N_TRIALS = 100  # 最大试验次数
N_JOBS = 1 # 并行运行的任务数
N_STARTUP_TRIALS = 5  # 在 N_STARTUP_TRIALS 次试验后停止随机采样
N_EVALUATIONS = 2  # 训练过程中的评估次数
N_TIMESTEPS = int(2e4)  # 训练预算
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5 # 评估环境的数量
N_EVAL_EPISODES = 10 # 每次评估的回合数
TIMEOUT = int(60 * 15)  # 15 分钟超时

ENV_ID = "CartPole-v1"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "env": ENV_ID,
}

### 练习 (5分钟): 定义搜索空间

In [ ]:
from typing import Any, Dict
import torch
import torch.nn as nn

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    A2C 超参数的采样器。

    :param trial: Optuna 的 trial 对象
    :return: 为给定 trial 采样的超参数。
    """
    # 折扣因子在 0.9 到 0.9999 之间
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    # 8, 16, 32, ... 1024
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)

    ### 这里是练习的答案 ###
    # - 定义学习率的搜索空间 [1e-5, 1] (对数尺度) -> `suggest_float`
    # - 定义网络结构的搜索空间 ["tiny", "small"] -> `suggest_categorical`
    # - 定义激活函数的搜索空间 ["tanh", "relu"]
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    ### 答案结束 ###

    # 在 Optuna 的界面中显示真实值，方便查看
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]}
        if net_arch == "tiny"
        else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
        },
    }

### 定义目标函数 (Objective Function)

首先，我们定义一个自定义的回调函数（Callback），用于向 Optuna 报告周期性评估的结果：

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    用于评估和报告 trial 结果的回调函数。
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # 调用父类的方法来评估策略
            super()._on_step()
            self.eval_idx += 1
            # 向 Optuna 报告结果
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # 如果需要，剪枝 trial
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### 练习 (10分钟): 定义目标函数

然后我们定义目标函数，它负责采样超参数、创建模型，并将结果返回给 Optuna。

In [ ]:
def objective(trial: optuna.Trial) -> float:
    """
    Optuna 使用的目标函数，用于评估一组配置（即一组超参数）。
    
    给定一个 trial 对象，它会采样超参数，
    对其进行评估，并报告结果（训练后的平均回合奖励）。
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    
    ### 这里是练习的答案 ###
    # 1. 采样超参数并更新默认的关键字参数
    params = sample_a2c_params(trial)
    kwargs.update(params)

    # 创建强化学习模型
    model = A2C(**kwargs)

    # 2. 使用 `make_vec_env`、`ENV_ID` 和 `N_EVAL_ENVS` 创建用于评估的环境
    eval_envs = make_vec_env(ENV_ID, n_envs=N_EVAL_ENVS)

    # 3. 创建上面定义的 `TrialEvalCallback` 回调函数
    # 它会每隔 `EVAL_FREQ` 步使用 `N_EVAL_EPISODES` 来周期性地评估和报告性能
    eval_callback = TrialEvalCallback(
        eval_envs,
        trial,
        n_eval_episodes=N_EVAL_EPISODES,
        eval_freq=EVAL_FREQ,
        deterministic=True
    )

    ### 答案结束 ###

    nan_encountered = False
    try:
        # 训练模型
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # 有时，随机的超参数可能会产生 NaN (Not a Number)
        print(e)
        nan_encountered = True
    finally:
        # 释放内存
        model.env.close()
        eval_envs.close()

    # 告诉优化器该 trial 失败了
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

### 优化循环

In [ ]:
import torch as th

# 为了更快的训练，将 PyTorch 的线程数设置为 1
th.set_num_threads(1)
# 选择采样器，可以是 random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# 在使用最大预算的 1/3 之前不进行剪枝
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
)
# 创建 study 并开始超参数优化
# `direction` 设置为 'maximize' 因为我们的目标是最大化奖励
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print("完成的试验次数: ", len(study.trials))

print("最佳试验:")
trial = study.best_trial

print(f"  价值 (Value): {trial.value}")

print("  参数 (Params): ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  用户自定义属性 (User attrs):")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

# 保存研究结果
study.trials_dataframe().to_csv("study_results_a2c_cartpole.csv")

# 可视化结果
fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

一个更完整的示例可以在这里找到： https://github.com/DLR-RM/rl-baselines3-zoo

# 结论

在本 Notebook 中我们学到了：
- 好的超参数的重要性
- 如何使用 Optuna 进行自动超参数搜索
